In [ ]:
```xml
<VSCode.Cell language="markdown">
# Service Communication Patterns

Strategies for inter-service communication in microservices: synchronous vs asynchronous, protocols, and composition patterns.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    COMMUNICATION PATTERNS                                    │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│   SYNCHRONOUS                              ASYNCHRONOUS                      │
│   ───────────                              ────────────                      │
│                                                                              │
│   ┌─────────┐   Request   ┌─────────┐     ┌─────────┐         ┌─────────┐  │
│   │ Service │───────────▶│ Service │     │ Service │         │ Service │  │
│   │    A    │◀───────────│    B    │     │    A    │         │    B    │  │
│   └─────────┘   Response  └─────────┘     └────┬────┘         └────▲────┘  │
│                                                │                    │       │
│   - REST, gRPC, GraphQL                        ▼                    │       │
│   - Immediate response                    ┌─────────┐               │       │
│   - Simpler to implement                  │ Message │───────────────┘       │
│   - Tighter coupling                      │  Broker │                       │
│                                           └─────────┘                       │
│                                                                              │
│                                           - Events, Commands                 │
│                                           - Fire-and-forget                  │
│                                           - Decoupled, resilient             │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Synchronous vs Asynchronous Communication

| Aspect | Synchronous | Asynchronous |
|--------|-------------|--------------|
| **Coupling** | Temporal coupling (both must be up) | Decoupled in time |
| **Latency** | Cumulative across calls | Non-blocking |
| **Error Handling** | Immediate feedback | Delayed/complex |
| **Debugging** | Easier to trace | Harder (distributed) |
| **Scalability** | Limited by slowest service | Better isolation |
| **Consistency** | Easier strong consistency | Eventual consistency |
| **Use Cases** | Queries, validation | Events, workflows |

```
DECISION TREE:

  Need immediate response?
         │
    Yes  │  No
    ▼    │  ▼
  SYNC   │  Is ordering important?
         │       │
         │  Yes  │  No
         │  ▼    │  ▼
         │ QUEUE │ PUB/SUB
         │(Kafka)│ (SNS, RabbitMQ fanout)
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## REST API Communication

```
REST (Representational State Transfer)
─────────────────────────────────────

RESOURCE-ORIENTED:
  GET    /orders/123      → Read order
  POST   /orders          → Create order
  PUT    /orders/123      → Replace order
  PATCH  /orders/123      → Update order
  DELETE /orders/123      → Delete order

HTTP STATUS CODES:
┌────────┬───────────────────────────────────────────┐
│  2xx   │ Success (200 OK, 201 Created, 204 No Content)
├────────┼───────────────────────────────────────────┤
│  3xx   │ Redirect (301 Moved, 304 Not Modified)    │
├────────┼───────────────────────────────────────────┤
│  4xx   │ Client Error (400 Bad Request, 404 Not Found)
├────────┼───────────────────────────────────────────┤
│  5xx   │ Server Error (500 Internal, 503 Unavailable)
└────────┴───────────────────────────────────────────┘

BEST PRACTICES:
✓ Use nouns for resources, not verbs
✓ Version your API (/v1/orders)
✓ Use proper HTTP methods
✓ Return meaningful status codes
✓ Support pagination, filtering
✓ Use HATEOAS for discoverability
```
</VSCode.Cell>
<VSCode.Cell language="python">
# REST API Example with FastAPI

from fastapi import FastAPI, HTTPException, Query, status
from pydantic import BaseModel
from typing import List, Optional
from uuid import UUID, uuid4
from datetime import datetime
from enum import Enum

# === MODELS ===
class OrderStatus(str, Enum):
    PENDING = "pending"
    CONFIRMED = "confirmed"
    SHIPPED = "shipped"

class OrderItem(BaseModel):
    product_id: UUID
    quantity: int
    unit_price: float

class OrderCreate(BaseModel):
    customer_id: UUID
    items: List[OrderItem]

class Order(BaseModel):
    id: UUID
    customer_id: UUID
    items: List[OrderItem]
    status: OrderStatus
    total: float
    created_at: datetime
    
    # HATEOAS links
    class Config:
        schema_extra = {
            "example": {
                "id": "123e4567-e89b-12d3-a456-426614174000",
                "customer_id": "123e4567-e89b-12d3-a456-426614174001",
                "items": [{"product_id": "...", "quantity": 2, "unit_price": 29.99}],
                "status": "pending",
                "total": 59.98,
                "created_at": "2024-01-15T10:30:00Z"
            }
        }

class PaginatedOrders(BaseModel):
    items: List[Order]
    total: int
    page: int
    page_size: int
    links: dict

# === IN-MEMORY STORE ===
orders_db: dict[UUID, Order] = {}

# === API ===
app = FastAPI(title="Order Service", version="1.0.0")

@app.post("/v1/orders", response_model=Order, status_code=status.HTTP_201_CREATED)
async def create_order(order_data: OrderCreate):
    """Create a new order."""
    order_id = uuid4()
    total = sum(item.quantity * item.unit_price for item in order_data.items)
    
    order = Order(
        id=order_id,
        customer_id=order_data.customer_id,
        items=order_data.items,
        status=OrderStatus.PENDING,
        total=total,
        created_at=datetime.utcnow()
    )
    orders_db[order_id] = order
    return order

@app.get("/v1/orders/{order_id}", response_model=Order)
async def get_order(order_id: UUID):
    """Get order by ID."""
    if order_id not in orders_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Order {order_id} not found"
        )
    return orders_db[order_id]

@app.get("/v1/orders", response_model=PaginatedOrders)
async def list_orders(
    page: int = Query(1, ge=1),
    page_size: int = Query(10, ge=1, le=100),
    status: Optional[OrderStatus] = None
):
    """List orders with pagination and filtering."""
    filtered = list(orders_db.values())
    if status:
        filtered = [o for o in filtered if o.status == status]
    
    start = (page - 1) * page_size
    end = start + page_size
    
    return PaginatedOrders(
        items=filtered[start:end],
        total=len(filtered),
        page=page,
        page_size=page_size,
        links={
            "self": f"/v1/orders?page={page}&page_size={page_size}",
            "next": f"/v1/orders?page={page+1}&page_size={page_size}" if end < len(filtered) else None,
            "prev": f"/v1/orders?page={page-1}&page_size={page_size}" if page > 1 else None
        }
    )

@app.patch("/v1/orders/{order_id}/status")
async def update_order_status(order_id: UUID, new_status: OrderStatus):
    """Update order status."""
    if order_id not in orders_db:
        raise HTTPException(status_code=404, detail="Order not found")
    orders_db[order_id].status = new_status
    return {"status": "updated"}

print("REST API defined with FastAPI")
print("Endpoints: POST/GET /v1/orders, GET/PATCH /v1/orders/{id}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## gRPC Communication

```
gRPC (Google Remote Procedure Call)
───────────────────────────────────

CHARACTERISTICS:
┌────────────────────┬─────────────────────────────────────────┐
│ Protocol           │ HTTP/2 (multiplexing, streaming)       │
├────────────────────┼─────────────────────────────────────────┤
│ Serialization      │ Protocol Buffers (binary, compact)     │
├────────────────────┼─────────────────────────────────────────┤
│ Interface          │ Strongly typed, code generated         │
├────────────────────┼─────────────────────────────────────────┤
│ Performance        │ 2-10x faster than JSON/REST            │
├────────────────────┼─────────────────────────────────────────┤
│ Streaming          │ Unary, server, client, bidirectional   │
└────────────────────┴─────────────────────────────────────────┘

COMMUNICATION PATTERNS:

  UNARY                    SERVER STREAMING
  ─────                    ────────────────
  Client ──Request──▶      Client ──Request──▶
  Client ◀──Response──     Client ◀──Stream────
                           Client ◀────────────
                           Client ◀────────────

  CLIENT STREAMING         BIDIRECTIONAL
  ────────────────         ─────────────
  Client ──Stream──▶       Client ──Stream──▶
  Client ──────────▶       Client ◀──Stream──
  Client ◀──Response──     Client ──Stream──▶
                           Client ◀──Stream──
```
</VSCode.Cell>
<VSCode.Cell language="python">
# gRPC Proto Definition and Python Implementation

# First, define the .proto file (order_service.proto):
proto_definition = """
syntax = "proto3";

package orders;

service OrderService {
    // Unary RPC
    rpc GetOrder (GetOrderRequest) returns (Order);
    rpc CreateOrder (CreateOrderRequest) returns (Order);
    
    // Server streaming - watch order status changes
    rpc WatchOrder (WatchOrderRequest) returns (stream OrderStatusUpdate);
    
    // Client streaming - bulk order creation
    rpc CreateOrders (stream CreateOrderRequest) returns (BulkOrderResponse);
    
    // Bidirectional streaming - real-time order updates
    rpc OrderUpdates (stream OrderCommand) returns (stream OrderEvent);
}

message GetOrderRequest {
    string order_id = 1;
}

message CreateOrderRequest {
    string customer_id = 1;
    repeated OrderItem items = 2;
}

message OrderItem {
    string product_id = 1;
    int32 quantity = 2;
    double unit_price = 3;
}

message Order {
    string id = 1;
    string customer_id = 2;
    repeated OrderItem items = 3;
    OrderStatus status = 4;
    double total = 5;
}

enum OrderStatus {
    PENDING = 0;
    CONFIRMED = 1;
    SHIPPED = 2;
    DELIVERED = 3;
}

message OrderStatusUpdate {
    string order_id = 1;
    OrderStatus old_status = 2;
    OrderStatus new_status = 3;
    string timestamp = 4;
}

message WatchOrderRequest {
    string order_id = 1;
}

message BulkOrderResponse {
    int32 created_count = 1;
    repeated string order_ids = 2;
}

message OrderCommand {
    string order_id = 1;
    string action = 2;  // "cancel", "update", etc.
}

message OrderEvent {
    string order_id = 1;
    string event_type = 2;
    string payload = 3;
}
"""

print("=== gRPC Proto Definition ===")
print(proto_definition)
</VSCode.Cell>
<VSCode.Cell language="python">
# gRPC Server Implementation (simulated without actual gRPC dependencies)

from dataclasses import dataclass
from typing import Iterator, List
from datetime import datetime
from uuid import uuid4
import time

@dataclass
class Order:
    id: str
    customer_id: str
    items: list
    status: str
    total: float

class OrderServicer:
    """gRPC Service Implementation (simulated)."""
    
    def __init__(self):
        self.orders = {}
    
    def GetOrder(self, request):
        """Unary RPC: Get single order."""
        order_id = request.get("order_id")
        if order_id not in self.orders:
            raise Exception(f"Order {order_id} not found")  # Would be grpc.StatusCode.NOT_FOUND
        return self.orders[order_id]
    
    def CreateOrder(self, request) -> Order:
        """Unary RPC: Create order."""
        order_id = str(uuid4())
        order = Order(
            id=order_id,
            customer_id=request["customer_id"],
            items=request["items"],
            status="PENDING",
            total=sum(i["quantity"] * i["unit_price"] for i in request["items"])
        )
        self.orders[order_id] = order
        return order
    
    def WatchOrder(self, request) -> Iterator[dict]:
        """Server Streaming: Watch order status updates."""
        order_id = request.get("order_id")
        
        # Simulate status updates
        statuses = ["PENDING", "CONFIRMED", "SHIPPED", "DELIVERED"]
        for i, new_status in enumerate(statuses[1:], 1):
            time.sleep(0.1)  # Simulated delay
            yield {
                "order_id": order_id,
                "old_status": statuses[i-1],
                "new_status": new_status,
                "timestamp": datetime.utcnow().isoformat()
            }
    
    def CreateOrders(self, requests: Iterator[dict]) -> dict:
        """Client Streaming: Bulk create orders."""
        order_ids = []
        for request in requests:
            order = self.CreateOrder(request)
            order_ids.append(order.id)
        return {"created_count": len(order_ids), "order_ids": order_ids}
    
    def OrderUpdates(self, commands: Iterator[dict]) -> Iterator[dict]:
        """Bidirectional Streaming: Real-time updates."""
        for cmd in commands:
            # Process command and yield event
            yield {
                "order_id": cmd["order_id"],
                "event_type": f"{cmd['action']}_processed",
                "payload": f"Command {cmd['action']} executed"
            }

# === DEMO ===
service = OrderServicer()

# Create order
order = service.CreateOrder({
    "customer_id": "cust-123",
    "items": [{"product_id": "prod-1", "quantity": 2, "unit_price": 29.99}]
})
print(f"Created: {order}")

# Watch order updates (server streaming)
print("\nServer Streaming - Order Updates:")
for update in service.WatchOrder({"order_id": order.id}):
    print(f"  {update['old_status']} -> {update['new_status']}")

# Client streaming - bulk create
print("\nClient Streaming - Bulk Create:")
orders_to_create = [
    {"customer_id": "cust-1", "items": [{"product_id": "p1", "quantity": 1, "unit_price": 10}]},
    {"customer_id": "cust-2", "items": [{"product_id": "p2", "quantity": 2, "unit_price": 20}]},
]
result = service.CreateOrders(iter(orders_to_create))
print(f"  Created {result['created_count']} orders")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## GraphQL Communication

```
GraphQL
───────

CHARACTERISTICS:
- Client specifies exactly what data it needs
- Single endpoint (typically /graphql)
- Strongly typed schema
- No over-fetching or under-fetching

QUERY EXAMPLE:
┌─────────────────────────────────────────────────────────────────────────────┐
│  query {                           │  {                                    │
│    order(id: "123") {              │    "order": {                         │
│      id                            │      "id": "123",                     │
│      status                        │      "status": "SHIPPED",             │
│      customer {                    │      "customer": {                    │
│        name                        │        "name": "John Doe"             │
│        email                       │      },                               │
│      }                             │      "items": [                       │
│      items {                       │        {                              │
│        product { name }            │          "product": { "name": "Widget"}│
│        quantity                    │          "quantity": 2                │
│      }                             │        }                              │
│    }                               │      ]                                │
│  }                                 │    }                                  │
│                                    │  }                                    │
│  ─────────────────────             │  ─────────────────────────            │
│  Request (exactly what needed)     │  Response (no extra fields)          │
└─────────────────────────────────────────────────────────────────────────────┘

FEDERATION (Multiple Services):
  ┌─────────────────────────────────────────────────────────────┐
  │                    GraphQL Gateway                          │
  │              (Federation / Stitching)                       │
  └──────────────────────────┬──────────────────────────────────┘
                             │
       ┌─────────────────────┼─────────────────────┐
       ▼                     ▼                     ▼
  ┌─────────────┐     ┌─────────────┐      ┌─────────────┐
  │   Orders    │     │   Users     │      │  Products   │
  │   Subgraph  │     │   Subgraph  │      │   Subgraph  │
  └─────────────┘     └─────────────┘      └─────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="python">
# GraphQL with Strawberry (Python GraphQL library)

import strawberry
from typing import List, Optional
from dataclasses import dataclass
from uuid import UUID, uuid4
from enum import Enum

# === TYPES ===
@strawberry.enum
class OrderStatus(Enum):
    PENDING = "pending"
    CONFIRMED = "confirmed"
    SHIPPED = "shipped"

@strawberry.type
class Customer:
    id: strawberry.ID
    name: str
    email: str

@strawberry.type
class Product:
    id: strawberry.ID
    name: str
    price: float

@strawberry.type
class OrderItem:
    product: Product
    quantity: int

@strawberry.type
class Order:
    id: strawberry.ID
    status: OrderStatus
    customer: Customer
    items: List[OrderItem]
    total: float

# === IN-MEMORY DATA ===
customers = {
    "c1": Customer(id="c1", name="John Doe", email="john@example.com"),
    "c2": Customer(id="c2", name="Jane Smith", email="jane@example.com"),
}

products = {
    "p1": Product(id="p1", name="Widget", price=29.99),
    "p2": Product(id="p2", name="Gadget", price=49.99),
}

orders = {
    "o1": {
        "id": "o1",
        "status": OrderStatus.SHIPPED,
        "customer_id": "c1",
        "items": [{"product_id": "p1", "quantity": 2}],
    }
}

# === RESOLVERS ===
def resolve_order(order_id: str) -> Optional[Order]:
    if order_id not in orders:
        return None
    o = orders[order_id]
    items = [
        OrderItem(
            product=products[i["product_id"]], 
            quantity=i["quantity"]
        ) for i in o["items"]
    ]
    return Order(
        id=o["id"],
        status=o["status"],
        customer=customers[o["customer_id"]],
        items=items,
        total=sum(products[i["product_id"]].price * i["quantity"] for i in o["items"])
    )

@strawberry.type
class Query:
    @strawberry.field
    def order(self, id: strawberry.ID) -> Optional[Order]:
        """Get order by ID - clients choose which fields to fetch."""
        return resolve_order(id)
    
    @strawberry.field
    def orders(self, status: Optional[OrderStatus] = None) -> List[Order]:
        """List orders with optional filtering."""
        result = []
        for order_id in orders:
            order = resolve_order(order_id)
            if order and (status is None or order.status == status):
                result.append(order)
        return result

@strawberry.input
class CreateOrderInput:
    customer_id: strawberry.ID
    product_ids: List[strawberry.ID]
    quantities: List[int]

@strawberry.type  
class Mutation:
    @strawberry.mutation
    def create_order(self, input: CreateOrderInput) -> Order:
        """Create a new order."""
        order_id = f"o{len(orders) + 1}"
        orders[order_id] = {
            "id": order_id,
            "status": OrderStatus.PENDING,
            "customer_id": input.customer_id,
            "items": [
                {"product_id": pid, "quantity": qty}
                for pid, qty in zip(input.product_ids, input.quantities)
            ]
        }
        return resolve_order(order_id)

# Create schema
schema = strawberry.Schema(query=Query, mutation=Mutation)

# Example queries
print("=== GraphQL Schema Defined ===\n")
print("Example Query:")
print("""
query {
  order(id: "o1") {
    id
    status
    customer { name email }
    items { product { name } quantity }
    total
  }
}
""")

# Simulate query execution
result = resolve_order("o1")
print(f"Result: {result}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Protocol Comparison

| Feature | REST | gRPC | GraphQL |
|---------|------|------|---------|
| **Protocol** | HTTP/1.1 or 2 | HTTP/2 | HTTP |
| **Format** | JSON (text) | Protobuf (binary) | JSON |
| **Contract** | OpenAPI/Swagger | Proto files | SDL Schema |
| **Performance** | Good | Excellent (2-10x) | Good |
| **Streaming** | Polling/WebSocket | Native | Subscriptions |
| **Browser Support** | Native | Needs proxy | Native |
| **Learning Curve** | Low | Medium | Medium |
| **Over-fetching** | Common | Minimal | Never |
| **Caching** | HTTP caching | Complex | Query-based |
| **Best For** | Public APIs | Internal services | BFF, mobile |
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Message Brokers & Async Communication

```
MESSAGE BROKER PATTERNS:

1. POINT-TO-POINT (Queue)           2. PUB/SUB (Topic)
   ─────────────────────               ─────────────────

   Producer ────▶ [Queue] ────▶ Consumer     Publisher
                                                │
   - One consumer per message                   ▼
   - Load balancing                        [Topic]
   - Work distribution                    /   │   \
                                         ▼    ▼    ▼
                                       Sub1  Sub2  Sub3

                                     - All subscribers get copy
                                     - Fan-out pattern
                                     - Event broadcasting

POPULAR BROKERS:
┌─────────────┬─────────────────────────────────────────────────────────┐
│ Apache Kafka│ High throughput, ordered, persistent log               │
├─────────────┼─────────────────────────────────────────────────────────┤
│ RabbitMQ    │ Feature-rich, flexible routing, AMQP                   │
├─────────────┼─────────────────────────────────────────────────────────┤
│ AWS SQS     │ Managed queue, simple, scalable                        │
├─────────────┼─────────────────────────────────────────────────────────┤
│ AWS SNS     │ Managed pub/sub, integrates with SQS, Lambda           │
├─────────────┼─────────────────────────────────────────────────────────┤
│ Redis Pub/Sub│ Fast, ephemeral (no persistence)                      │
└─────────────┴─────────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="python">
# Event-Driven Communication Patterns

from dataclasses import dataclass, field
from datetime import datetime
from typing import Callable, Dict, List, Any
from uuid import uuid4
from abc import ABC, abstractmethod
import json

# === DOMAIN EVENTS ===
@dataclass
class DomainEvent:
    event_id: str = field(default_factory=lambda: str(uuid4()))
    occurred_at: datetime = field(default_factory=datetime.utcnow)
    
    def to_dict(self) -> dict:
        return {
            "event_id": self.event_id,
            "event_type": self.__class__.__name__,
            "occurred_at": self.occurred_at.isoformat(),
            "payload": {k: v for k, v in self.__dict__.items() 
                       if k not in ("event_id", "occurred_at")}
        }

@dataclass
class OrderPlaced(DomainEvent):
    order_id: str = ""
    customer_id: str = ""
    total: float = 0.0

@dataclass 
class OrderShipped(DomainEvent):
    order_id: str = ""
    tracking_number: str = ""

@dataclass
class PaymentCompleted(DomainEvent):
    order_id: str = ""
    amount: float = 0.0
    payment_method: str = ""

# === MESSAGE BROKER (In-Memory Simulation) ===
class MessageBroker:
    """Simulates a pub/sub message broker."""
    
    def __init__(self):
        self.subscribers: Dict[str, List[Callable]] = {}
        self.dead_letter_queue: List[dict] = []
    
    def subscribe(self, event_type: str, handler: Callable):
        """Subscribe to an event type."""
        if event_type not in self.subscribers:
            self.subscribers[event_type] = []
        self.subscribers[event_type].append(handler)
        print(f"[BROKER] Handler subscribed to {event_type}")
    
    def publish(self, event: DomainEvent):
        """Publish event to all subscribers."""
        event_type = event.__class__.__name__
        event_data = event.to_dict()
        
        print(f"[BROKER] Publishing {event_type}: {event.event_id[:8]}...")
        
        handlers = self.subscribers.get(event_type, [])
        if not handlers:
            print(f"[BROKER] No subscribers for {event_type}")
            return
        
        for handler in handlers:
            try:
                handler(event_data)
            except Exception as e:
                print(f"[BROKER] Handler failed: {e}")
                self.dead_letter_queue.append({
                    "event": event_data,
                    "error": str(e)
                })

# === SERVICE HANDLERS ===
class InventoryService:
    """Handles stock reservation when order placed."""
    
    def __init__(self, broker: MessageBroker):
        broker.subscribe("OrderPlaced", self.handle_order_placed)
    
    def handle_order_placed(self, event: dict):
        order_id = event["payload"]["order_id"]
        print(f"[INVENTORY] Reserving stock for order {order_id}")
        # Simulate stock reservation logic

class NotificationService:
    """Sends notifications for various events."""
    
    def __init__(self, broker: MessageBroker):
        broker.subscribe("OrderPlaced", self.handle_order_placed)
        broker.subscribe("OrderShipped", self.handle_order_shipped)
    
    def handle_order_placed(self, event: dict):
        order_id = event["payload"]["order_id"]
        print(f"[NOTIFY] Sending order confirmation for {order_id}")
    
    def handle_order_shipped(self, event: dict):
        order_id = event["payload"]["order_id"]
        tracking = event["payload"]["tracking_number"]
        print(f"[NOTIFY] Sending shipping notification: {order_id}, tracking: {tracking}")

class PaymentService:
    """Processes payments and emits completion events."""
    
    def __init__(self, broker: MessageBroker):
        self.broker = broker
        broker.subscribe("OrderPlaced", self.handle_order_placed)
    
    def handle_order_placed(self, event: dict):
        order_id = event["payload"]["order_id"]
        total = event["payload"]["total"]
        print(f"[PAYMENT] Processing payment of ${total} for {order_id}")
        
        # Simulate payment processing, then emit completion
        completion = PaymentCompleted(
            order_id=order_id,
            amount=total,
            payment_method="credit_card"
        )
        self.broker.publish(completion)

# === DEMO ===
print("=== Event-Driven Architecture Demo ===\n")

# Set up broker and services
broker = MessageBroker()
inventory_svc = InventoryService(broker)
notify_svc = NotificationService(broker)
payment_svc = PaymentService(broker)

print()

# Simulate order placement
order_event = OrderPlaced(
    order_id="order-12345",
    customer_id="cust-999",
    total=99.99
)
broker.publish(order_event)

print()

# Simulate shipping
ship_event = OrderShipped(
    order_id="order-12345",
    tracking_number="1Z999AA10123456784"
)
broker.publish(ship_event)
</VSCode.Cell>
<VSCode.Cell language="markdown">
## API Composition Patterns

```
PROBLEM: Client needs data from multiple services

BAD: Client makes multiple calls       GOOD: Backend aggregation
─────────────────────────────          ───────────────────────────

┌──────────┐                          ┌──────────┐
│  Client  │                          │  Client  │
└────┬─────┘                          └────┬─────┘
     │                                     │
     ├───▶ Order Service                   ▼
     ├───▶ User Service              ┌───────────┐
     ├───▶ Product Service           │ Aggregator│ ◀── Single call
     └───▶ Payment Service           │  (BFF)    │
                                     └─────┬─────┘
     Many round trips!                     │
     N+1 problem                     ┌─────┴─────┐
                                     ▼     ▼     ▼
                                   Orders Users Products


API COMPOSITION PATTERNS:
┌────────────────────┬────────────────────────────────────────────────────┐
│ BFF (Backend for   │ Dedicated backend per client type (web, mobile)   │
│ Frontend)          │ Optimizes payloads for each client                │
├────────────────────┼────────────────────────────────────────────────────┤
│ API Gateway        │ Single entry point, routes to services            │
│ Aggregation        │ Can compose responses from multiple services      │
├────────────────────┼────────────────────────────────────────────────────┤
│ GraphQL Federation │ Unified graph across multiple services            │
│                    │ Client queries, gateway resolves                  │
├────────────────────┼────────────────────────────────────────────────────┤
│ Parallel Calls     │ Make service calls in parallel, merge results     │
│ + Merge            │ Reduces latency vs sequential                     │
└────────────────────┴────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="python">
# API Composition: BFF Pattern

import asyncio
from dataclasses import dataclass
from typing import Dict, Any, Optional
from concurrent.futures import ThreadPoolExecutor
import time

@dataclass
class OrderDetails:
    """Composed response for client."""
    order_id: str
    status: str
    customer_name: str
    customer_email: str
    products: list
    payment_status: str
    total: float

class OrderServiceClient:
    async def get_order(self, order_id: str) -> dict:
        await asyncio.sleep(0.05)  # Simulate network
        return {
            "id": order_id,
            "status": "shipped",
            "customer_id": "c123",
            "product_ids": ["p1", "p2"],
            "total": 149.98
        }

class UserServiceClient:
    async def get_user(self, user_id: str) -> dict:
        await asyncio.sleep(0.03)
        return {
            "id": user_id,
            "name": "John Doe",
            "email": "john@example.com"
        }

class ProductServiceClient:
    async def get_products(self, product_ids: list) -> list:
        await asyncio.sleep(0.04)
        products = {
            "p1": {"id": "p1", "name": "Widget", "price": 49.99},
            "p2": {"id": "p2", "name": "Gadget", "price": 99.99}
        }
        return [products.get(pid, {}) for pid in product_ids]

class PaymentServiceClient:
    async def get_payment_status(self, order_id: str) -> dict:
        await asyncio.sleep(0.02)
        return {"order_id": order_id, "status": "completed"}

class OrderDetailsBFF:
    """
    Backend for Frontend: Composes data from multiple services
    into a single response optimized for the client.
    """
    
    def __init__(self):
        self.order_client = OrderServiceClient()
        self.user_client = UserServiceClient()
        self.product_client = ProductServiceClient()
        self.payment_client = PaymentServiceClient()
    
    async def get_order_details(self, order_id: str) -> OrderDetails:
        """
        Parallel composition: fetch from all services concurrently.
        """
        start = time.time()
        
        # First get order to know customer_id and product_ids
        order = await self.order_client.get_order(order_id)
        
        # Then fetch related data in parallel
        user_task = self.user_client.get_user(order["customer_id"])
        products_task = self.product_client.get_products(order["product_ids"])
        payment_task = self.payment_client.get_payment_status(order_id)
        
        user, products, payment = await asyncio.gather(
            user_task, products_task, payment_task
        )
        
        elapsed = (time.time() - start) * 1000
        print(f"Composed response in {elapsed:.0f}ms")
        
        return OrderDetails(
            order_id=order["id"],
            status=order["status"],
            customer_name=user["name"],
            customer_email=user["email"],
            products=products,
            payment_status=payment["status"],
            total=order["total"]
        )

# === DEMO ===
async def main():
    bff = OrderDetailsBFF()
    details = await bff.get_order_details("order-123")
    
    print("\n=== Composed Order Details ===")
    print(f"Order ID: {details.order_id}")
    print(f"Status: {details.status}")
    print(f"Customer: {details.customer_name} ({details.customer_email})")
    print(f"Products: {[p['name'] for p in details.products]}")
    print(f"Payment: {details.payment_status}")
    print(f"Total: ${details.total}")

# Run async demo
asyncio.run(main())
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Communication Anti-Patterns

```
ANTI-PATTERN 1: Chatty Communication
────────────────────────────────────
  ✗ BAD                              ✓ GOOD
  
  get_order()                        get_order_with_items()
  get_order_items()                  
  get_order_shipping()               Single call, complete data
  get_order_payment()


ANTI-PATTERN 2: Synchronous Chains
──────────────────────────────────
  ✗ BAD: Sync chain                  ✓ GOOD: Async events

  A ──sync──▶ B ──sync──▶ C          A ──event──▶ [Broker]
                                                    │
  - Cascading failures               B ◀─────────────┤
  - High latency                     C ◀─────────────┘


ANTI-PATTERN 3: Distributed Monolith
────────────────────────────────────
  ✗ Services tightly coupled         ✓ Loosely coupled
  
  - Shared database                  - Database per service
  - Same deploy cycle                - Independent deploys
  - Cross-service transactions       - Eventual consistency


ANTI-PATTERN 4: Temporal Coupling
─────────────────────────────────
  ✗ Both must be up                  ✓ Message queue buffers

  A ──────────▶ B (down!)            A ──▶ [Queue] ──▶ B
  Request fails!                     Works when B recovers
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Communication Decision Matrix

```
┌────────────────────────────────────────────────────────────────────────────┐
│                    WHEN TO USE WHAT                                         │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  SYNCHRONOUS (REST/gRPC)          │  ASYNCHRONOUS (Events/Messages)        │
│  ─────────────────────            │  ────────────────────────────          │
│                                   │                                         │
│  ✓ Need immediate response        │  ✓ Fire-and-forget                     │
│  ✓ Query operations               │  ✓ Long-running processes              │
│  ✓ Validation required            │  ✓ Event broadcasting                  │
│  ✓ Low latency critical           │  ✓ Decoupling required                 │
│  ✓ Simple request/response        │  ✓ Reliability over speed              │
│                                   │  ✓ Handling spikes                     │
│                                   │                                         │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  REST                    │  gRPC               │  GraphQL                  │
│  ────                    │  ────               │  ───────                  │
│  ✓ Public APIs           │  ✓ Internal services│  ✓ Complex UIs            │
│  ✓ Web browsers          │  ✓ High performance │  ✓ Mobile apps            │
│  ✓ Broad compatibility   │  ✓ Streaming needed │  ✓ Multiple clients       │
│  ✓ Simple CRUD           │  ✓ Polyglot services│  ✓ Reduce over-fetching   │
│                                                                             │
└────────────────────────────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Summary: Key Takeaways

| Principle | Recommendation |
|-----------|----------------|
| **Default to Async** | Use events for inter-service communication when possible |
| **Sync for Queries** | Synchronous calls are fine for read operations |
| **Minimize Chatter** | Reduce inter-service calls; batch when possible |
| **Use BFF** | Create aggregation layers for complex UIs |
| **gRPC Internal** | Use gRPC for high-performance internal calls |
| **REST External** | REST for public APIs and broad compatibility |
| **Event Sourcing** | Consider for audit trails and complex workflows |
| **Contract First** | Define APIs/events before implementation |
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Further Reading

- **Topics in this Knowledge Base:**
  - [Microservices Overview](./00_overview.ipynb)
  - [API Gateway & Service Mesh](./03_api_gateway_service_mesh.ipynb)
  - [Distributed Patterns](./04_distributed_patterns.ipynb)

- **External Resources:**
  - [gRPC Documentation](https://grpc.io/)
  - [GraphQL Specification](https://graphql.org/)
  - [Enterprise Integration Patterns](https://www.enterpriseintegrationpatterns.com/)
</VSCode.Cell>
```